In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [3]:
from urllib.request import urlopen
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import re

In [4]:
gpath = os.path.join('/content','drive', 'My Drive', '_data')

In [5]:
# endpoints available https://nsgreg.nga.mil/genc/registers.jsp
url = "https://nsgreg-api.nga.mil/geo-political/GENC/3/now"
resp = urlopen(url)

In [6]:
# parse xml data
tree = ET.parse(resp)
root = tree.getroot()

In [7]:
# all the child tags
[(i.tag, i.attrib) for i in root][:10]

[('{http://api.nsgreg.nga.mil/schema/genc/3.0}authority', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}baseline', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}promulgationDate', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {}),
 ('{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry', {})]

In [8]:
# let's look at all the elements in the tree
[elem.tag for elem in root.iter()][:30]

['{http://api.nsgreg.nga.mil/schema/genc/3.0}GENCStandardBaseline',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0}authority',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0}baseline',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0}promulgationDate',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0}GeopoliticalEntityEntry',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0}encoding',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char3Code',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char3CodeURISet',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}codespaceURL',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}codespaceURN',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}codespaceURNBased',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}codespaceURNBasedShort',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char2Code',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char2CodeURISet',
 '{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}codespa

In [9]:
# get the data
trigraphs = [trigraph.text for trigraph in root.iter('{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char3Code')]
digraphs = [digraph.text for digraph in root.iter('{http://api.nsgreg.nga.mil/schema/genc/3.0/genc-cmn}char2Code')]
shortnames = [shortname.text for shortname in root.iter('{http://api.nsgreg.nga.mil/schema/genc/3.0}shortName')]

In [10]:
# quick check of lengths of the lists
[len(i) for i in [trigraphs, digraphs, shortnames]]

[284, 284, 284]

In [11]:
# put lists into dataframe
df = pd.DataFrame({
    "Country": shortnames,
    "Digraph": digraphs,
    "Trigraph": trigraphs})

In [12]:
df.head()

,Country,Digraph,Trigraph
0,Afghanistan,AF,AFG
1,Akrotiri,QZ,XQZ
2,Åland Islands,AX,ALA
3,Albania,AL,ALB
4,Algeria,DZ,DZA


In [17]:
df.to_csv(os.path.join(gpath, 'country_refdata.csv'), index = None, encoding = 'utf-8')